# Cross-collection evaluation of SVM baseline in the MTG-Jamendo test set


In [6]:
%pylab inline
%load_ext autoreload

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import csv
import collections
import json
import os

## Evaluation strategies
We consider different strategies for evaluation. These strategies are concerned with the mapping between the classes defined by the classifiers and the classes defined in our generated ground truth. A direct mapping is not possible for any of our classifiers.

We want to know classifier performance given all music in a dataset, as well as its performance when only given the subset of music that it is aimed for:
* __S1__ Use only recordings whose ground truth has a matching class in the classifier. For example, if a recording is only annotated with the class electronic, and this class does not appear in the classifier model, we discard it.
* __S2__ Use all recordings in the ground truth and treat recordings from classes that do not exist in the model as an incorrect classification.

Where the ground truth has more than one class for a track we consider different methods of matching these classes to classifiers’ estimates:
 * __ONLY__: Only use recordings that have one ground-truth genre tag, and discard the rest of the recordings when computing evaluation metrics
 * __ALL__: When a recording has more than one ground-truth genre, accept an estimate as correct if it matches any of these, even though for the rest of the genres it could be considered a misclassification.


### Duplicates
As we have observed that duplicate recordings submitted to AcousticBrainz may have inconsistent genre es-
timates as a result of different low-level descriptors, we consider two ways of dealing with them:
 * __D1__ Remove all recordings that have duplicates from consideration
 * __D2__ Treat all instances (recordings) in our data as independent, even if they are the same song.


# Mappings

The rosamerica -> our ground truth mapping. We discard the speech class as it has no match in our ground truth

In [8]:
rosamerica_gt_map = {
    'cla': ['classical'],
    'dan': ['dance'], # We listened to this dataset. It's all eurotrance (subgenre of electronic)
    'hip': ['hip hop'],
    'jaz': ['jazz'],
    'pop': ['pop'],
    'rhy': ['rhythm and blues'],
    'roc': ['rock'],
}


Tzanetakis groundtruth mapping. We discard disco as there is no clear match to our ground truth (the class in the dataset contains both electronic disco and non-electronic funk/soul-type disco. We are not sure if we should match it to electronic or rhythm and blues. We don't have a separate top-level genre for metal so we map both metal and rock to rock.

In [9]:
tzanetakis_gt_map = {
    'blu': ['blues'],
    'cla': ['classical'],
    'cou': ['country'],
    'dis': ['disco'], #disco - electronic & acoustic soul
    'hip': ['hip hop'],
    'jaz': ['jazz'],
    'met': ['metal'],
    'pop': ['pop'],
    'reg': ['reggae'], # reggae, in our gt a subgenre of ska
    'roc': ['rock']
}

Dortmund groundtruth mapping. Both alternative and rock are mapped to rock. folkcountry is mapped to both folk and country.

In [10]:
dortmund_gt_map = {
    'alternative': ['alternative rock'], # alternative rock is a subgenre of rock
    'blues': ['blues'],
    'electronic': ['electronic'],
    'folkcountry': ['country'],
    'funksoulrnb': ['rhythm and blues'],
    'jazz': ['jazz'],
    'pop': ['pop'],
    'raphiphop': ['hip hop'],
    'rock': ['rock']
}


## Evaluation

In [12]:
import genreeval

ground_truth = "jamendo-genres-clean.json"

ros = ("Rosamerica", 'results_jamendo_svm/genre_rosamerica.json', rosamerica_gt_map)
dor = ("Dortmund", 'results_jamendo_svm/genre_dortmund.json', dortmund_gt_map)
tza = ("Tzanetakis", 'results_jamendo_svm/genre_tzanetakis.json', tzanetakis_gt_map)

res = {}
for name, filename, gtmap in [ros, dor, tza]:
    res[name] = genreeval.run_evaluations(ground_truth, gtmap, filename, name, "Evaluation")[name]

RUNNING EVALUATION: Rosamerica S1 ALL D1
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S1 - ALL - D1
 [                  0%                  ]

RUNNING EVALUATION: Rosamerica S1 ALL D2
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S1 - ALL - D2
 [                  0%                  ]

RUNNING EVALUATION: Rosamerica S1 ONLY D1
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S1 - ONLY - D1
 [                  0%                  ]

RUNNING EVALUATION: Rosamerica S1 ONLY D2
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S1 - ONLY - D2
 [                  0%                  ]

RUNNING EVALUATION: Rosamerica S2 ALL D1
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S2 - ALL - D1
 [                  0%                  ]

RUNNING EVALUATION: Rosamerica S2 ALL D2
Loading estimates...
done
Loading ground truth...
done
Rosamerica - S2 - ALL - D2
 [                  0%                  ]



In [13]:
output = ""
from IPython.display import HTML
for classifier in ["Rosamerica", "Dortmund", "Tzanetakis"]:
    output += "<h1>%s</h1>" % classifier
    for strat in ["S1", "S2"]:
        for var in ["ONLY", "ALL"]: #"FIRST"
            for d in ["D1", "D2"]:
                results = res[classifier][strat][var][d]
                results.save("results")
                #fname = "%s.csv" % results.name.lower().replace(" ", "").replace(",", "-")
                #results.confusion_matrix.to_csv(fname)
                output += "<h2>%s</h2>\n" % (results.data['name'], )
                output += "<p><b>Raw accuracy: </b>%s\n" % results.raw_accuracy
                output += "<br><b>Normalised accuracy: </b>%s</p>\n" % results.normalised_accuracy
                output += results.html + "\n\n"
HTML(output)

,classical,dance,hip hop,jazz,pop,rhythm and blues,rock
classical,63.74,1.71,3.96,6.74,6.1,13.58,4.17
dance,0.97,51.78,15.53,0.65,7.44,21.04,2.59
hip hop,1.04,21.3,38.7,0.78,15.32,20.52,2.34
jazz,9.2,5.2,4.8,25.6,11.2,41.2,2.8
pop,10.66,8.84,4.03,4.55,27.18,40.83,3.9
rhythm and blues,2.78,19.44,12.5,4.17,19.44,37.5,4.17
rock,2.04,6.64,2.43,2.17,26.05,18.9,41.76
,classical,dance,hip hop,jazz,pop,rhythm and blues,rock
classical,63.74,1.71,3.96,6.74,6.1,13.58,4.17
dance,0.97,51.78,15.53,0.65,7.44,21.04,2.59
